<a href="https://colab.research.google.com/github/oscartg25/Yelp_GoogleMaps_ProyectoFinal/blob/master/ML_Text_personalized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai


In [36]:
!pip install textblob


In [38]:
import pandas as pd
import numpy as np
import openai  # Importamos la biblioteca OpenAI
import spacy
from textblob import TextBlob


In [4]:
#cargamos la data
df_rating = pd.read_csv('text_rating.csv')

In [28]:
# Configurar tu API Key de GooseAI
openai.api_key = "sk-AcM7DAh7ELJ7gTbE02KvlmKBJ5gXCIdH3nJ40G62sSwc1A3S"

openai.api_base = "https://api.goose.ai/v1"

In [29]:
# Obtener la lista de motores disponibles
engines = openai.Engine.list()

In [30]:
# Imprimir información sobre cada motor
for engine in engines.data:
    print("Engine ID:", engine.id)
    print("Engine Name:", engine.name)
    print("Engine Description:", engine.description)
    print("Tokenizer:", engine.tokenizer)
    print("Ready:", engine.ready)
    print("---")

Engine ID: cassandra-lit-2-8b
Engine Name: Cassandra 2.8B
Engine Description: A literary finetune based on the Pythia 2.8b model with about 400mb of texts from various science fiction, fantasy, fiction, horror, and interactive fiction. Great at chat!
Tokenizer: pile
Ready: True
---
Engine ID: cassandra-lit-6-9b
Engine Name: Cassandra 6.9B
Engine Description: A literary finetune based on the Pythia 2.8b model with about 400mb of texts from various science fiction, fantasy, fiction, horror, and interactive fiction. Great at chat!
Tokenizer: pile
Ready: True
---
Engine ID: convo-6b
Engine Name: Convo 6B
Engine Description: Convo-6B is a GPT-J 6B model fine-tuned on a collection of high quality open source datasets which amount to 6 million messages. The primary goal of the model is to provide improved performance and generalization when generating multi-turn dialogue for characters that were not present from within the fine tuning data.
Tokenizer: gpt2
Ready: True
---
Engine ID: fairseq-1

In [31]:
#Seleccionamos Motor
selected_engine_id = "gpt-neo-1-3b"

In [40]:
# Función para generar respuesta usando Goose.ai
def generate_response(comment):
    response = openai.Completion.create(
        engine="gpt-neo-1-3b",  # Reemplaza con el ID del motor que deseas utilizar
        prompt=comment,
        max_tokens=50  # Ajustar este valor según tus necesidades
    )
    return response.choices[0].text.strip()


In [41]:
# Función para analizar el sentimiento
def analyze_sentiment(text):
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity
    if sentiment > 0:
        return 5
    elif sentiment == 0:
        return 3
    else:
        return 1


In [42]:
# Iterar a través de los comentarios en el DataFrame
for index, row in df_rating.iterrows():
    comment = row['text']

    if isinstance(comment, str):  # Solo procesar comentarios no nulos
        generated_response = generate_response(comment)
        sentiment_label = analyze_sentiment(comment)

        df_rating.at[index, 'generated_response'] = generated_response
        df_rating.at[index, 'predicted_rating'] = sentiment_label

In [43]:
# Mostrar el DataFrame con las respuestas generadas y las predicciones de rating
print(df_rating)

                                                  text  rating  \
0    The lunch buffet was really good! Would defini...     NaN   
1                                   Sweet potato fries     NaN   
2    Ciders please! None on tap right now but next ...     NaN   
3    This has quickly become my new favorite restau...     NaN   
4                                          Madness....     NaN   
..                                                 ...     ...   
495                              Sit in the courtyard!     NaN   
496  Amazing ambience in a historic hotel.  Creativ...     NaN   
497  This is a great place to bring your little one...     NaN   
498  I loved my vegan pizza w/ teese!! Absolutely a...     NaN   
499                                  Yelp elite event!     NaN   

                                    generated_response sentiment_label  \
0    if the chance came up. Who knows I might claim...        positivo   
1    . Caramelized onions and butter served with gr...     

In [45]:
# Ingresar un texto personalizado para obtener el rating predicho
custom_text = "This restaurant is shit"
custom_sentiment_label = analyze_sentiment(custom_text)
custom_predicted_rating = custom_sentiment_label
print("Texto personalizado:", custom_text)
print("Análisis de sentimiento:", custom_sentiment_label)
print("Rating predicho:", custom_predicted_rating)

Texto personalizado: This restaurant is shit
Análisis de sentimiento: 1
Rating predicho: 1
